In [ ]:
!pip install -U "getdaft[deltalake]"
!pip install -q polars
!pip install -q azure-identity

# ***Authentifiction***

In [ ]:
from azure.identity import ClientSecretCredential, AuthenticationRequiredError
from google.colab import userdata
credential = ClientSecretCredential(
                client_id=userdata.get("appId"),
                client_secret=userdata.get("secret"),
                tenant_id=userdata.get ("tenantId")
                )
access_token =       credential.get_token("https://storage.azure.com/.default").token
storage_options=     {"bearer_token": access_token, "use_fabric_endpoint": "true"}
### lakehouse path is something like this : "abfss://workspace@onelake.dfs.fabric.microsoft.com/Lakehousename.Lakehouse/Tables/"
lakehouse =           userdata.get("table_path_onelake")

# ***Deltalake***

In [ ]:
from deltalake import DeltaTable
scada = DeltaTable(lakehouse+'scada',storage_options=storage_options).to_pyarrow_dataset()

In [ ]:
import duckdb
duckdb.sql("select * from scada limit 10").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,UNIT,VERSION,SETTLEMENTDATE,RUNNO,DUID,INTERVENTION,DISPATCHMODE,AGCSTATUS,INITIALMW,TOTALCLEARED,...,RAISEREGACTUALAVAILABILITY,LOWER6SECACTUALAVAILABILITY,LOWER60SECACTUALAVAILABILITY,LOWER5MINACTUALAVAILABILITY,LOWERREGACTUALAVAILABILITY,transactionId,file,PRIORITY,DATE,year
0,None,NaN,2024-03-25 00:00:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
1,None,NaN,2024-03-25 00:05:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
2,None,NaN,2024-03-25 00:10:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
3,None,NaN,2024-03-25 00:15:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
4,None,NaN,2024-03-25 00:20:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
5,None,NaN,2024-03-25 00:25:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
6,None,NaN,2024-03-25 00:30:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
7,None,NaN,2024-03-25 00:35:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
8,None,NaN,2024-03-25 00:40:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024
9,None,NaN,2024-03-25 00:45:00+00:00,NaN,SOUTH_CARDUP,0.0,NaN,NaN,0.107439,NaN,...,NaN,NaN,NaN,NaN,NaN,1c72db34-b3a0-4e0d-95c4-7dee4e7a3ec1,SCADA_2024-03-25.json,1,2024-03-25,2024


## ***Polars***

In [ ]:
import polars as pl
scada = pl.scan_delta(lakehouse+'scada', storage_options=storage_options)
x = scada.limit(10).collect()
x

In [ ]:
from deltalake.writer import write_deltalake
write_deltalake(lakehouse+'zzzz',x.to_arrow(),mode="overwrite",storage_options=storage_options)

# **Daft**

In [ ]:
%%time
import daft
from daft.io import IOConfig, AzureConfig
io_config = IOConfig(azure=AzureConfig(storage_account="onelake",endpoint_url="https://onelake.blob.fabric.microsoft.com",bearer_token=access_token))
df = daft.read_deltalake(lakehouse+'scada', io_config=io_config)
df

CPU times: user 255 ms, sys: 22.1 ms, total: 277 ms
Wall time: 4.73 s


UNITUtf8,VERSIONFloat64,"SETTLEMENTDATETimestamp(Microseconds, Some(""UTC""))",RUNNOFloat64,DUIDUtf8,INTERVENTIONFloat64,DISPATCHMODEFloat64,AGCSTATUSFloat64,INITIALMWFloat64,TOTALCLEAREDFloat64,RAMPDOWNRATEFloat64,RAMPUPRATEFloat64,LOWER5MINFloat64,LOWER60SECFloat64,LOWER6SECFloat64,RAISE5MINFloat64,RAISE60SECFloat64,RAISE6SECFloat64,MARGINAL5MINVALUEFloat64,MARGINAL60SECVALUEFloat64,MARGINAL6SECVALUEFloat64,MARGINALVALUEFloat64,VIOLATION5MINDEGREEFloat64,VIOLATION60SECDEGREEFloat64,VIOLATION6SECDEGREEFloat64,VIOLATIONDEGREEFloat64,LOWERREGFloat64,RAISEREGFloat64,AVAILABILITYFloat64,RAISE6SECFLAGSFloat64,RAISE60SECFLAGSFloat64,RAISE5MINFLAGSFloat64,RAISEREGFLAGSFloat64,LOWER6SECFLAGSFloat64,LOWER60SECFLAGSFloat64,LOWER5MINFLAGSFloat64,LOWERREGFLAGSFloat64,RAISEREGAVAILABILITYFloat64,RAISEREGENABLEMENTMAXFloat64,RAISEREGENABLEMENTMINFloat64,LOWERREGAVAILABILITYFloat64,LOWERREGENABLEMENTMAXFloat64,LOWERREGENABLEMENTMINFloat64,RAISE6SECACTUALAVAILABILITYFloat64,RAISE60SECACTUALAVAILABILITYFloat64,RAISE5MINACTUALAVAILABILITYFloat64,RAISEREGACTUALAVAILABILITYFloat64,LOWER6SECACTUALAVAILABILITYFloat64,LOWER60SECACTUALAVAILABILITYFloat64,LOWER5MINACTUALAVAILABILITYFloat64,LOWERREGACTUALAVAILABILITYFloat64,transactionIdUtf8,fileUtf8,PRIORITYInt32,DATEDate,yearInt32


In [ ]:
from daft import col
df.limit(10000).groupby('DUID').agg(
    col('INITIALMW').sum().alias("sum")
    ).show()

DUIDUtf8,sumFloat64
SOUTH_CARDUP,1063.742655459998
